In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import random
%config IPCompleter.greedy=True
os.listdir("C:\\Users\\chris\\OneDrive - Irwin's Safety and Industrial Labour Services Ltd\\Irwins\\output\\simulation")

['billable.csv',
 'cec_data.csv',
 'cec_data_tool_time.csv',
 'company_score.csv',
 'islam_data.csv',
 'islam_data2.csv',
 'rfid_simulation.csv',
 'workers.csv',
 'work_coverage.csv']

In [2]:
path = "C:\\Users\\chris\\OneDrive - Irwin's Safety and Industrial Labour Services Ltd\\Irwins\\output\\simulation"

In [3]:
workers = pd.read_csv(path+"\\workers.csv")
events = pd.read_csv(path+"\\rfid_simulation.csv",parse_dates=['TimeIn','TimeOut'])
cec_2018 = pd.read_csv(path+"\\cec_data.csv",
                       parse_dates=['clean_date','start_time','stop_time','work_start'])

In [4]:
company_score = cec_2018.merge(events.merge(workers, left_on='WorkerId',right_on='workerid',how='left'),
                          left_on='index_id',right_on='index_id',how='left')
company_score.head()

,index_id,csas_id,clean_date,entry_name,shift,crew_size,duration,facility_owner,proj_name,proj_year,...,vessel_clean,vessel_entry,eventid,WorkerId,TimeIn,TimeOut,workerid,FirstName,LastName,Company
0,19910,9363945241,2018-05-26,Door 51,Night,2.0,1.833333,Enmax,Calgary Energy Centre,2018,...,CT Generator,CT Generator: Door 51,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,19911,9363945241,2018-05-26,Door 51,Night,2.0,0.050000,Enmax,Calgary Energy Centre,2018,...,CT Generator,CT Generator: Door 51,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,19912,9363945241,2018-05-26,Door 51,Night,2.0,0.050000,Enmax,Calgary Energy Centre,2018,...,CT Generator,CT Generator: Door 51,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,19913,9363945241,2018-05-26,Door 51,Night,2.0,1.066667,Enmax,Calgary Energy Centre,2018,...,CT Generator,CT Generator: Door 51,0.0,0.0,2018-05-26 20:26:00,2018-05-26 21:30:00,0.0,Brian,Mage,MillwrightCorp1
4,19914,9363945241,2018-05-26,Door 51,Night,2.0,0.000000,Enmax,Calgary Energy Centre,2018,...,CT Generator,CT Generator: Door 51,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [5]:
company_score['worked_hours'] = ((company_score.TimeOut-company_score.TimeIn).
                                 astype('timedelta64[s]')/3600)
company_score.groupby(['Company','csas_id'])['worked_hours'].sum()

Company       csas_id   
LaborerCorp   16064865      1.816944
              47431840      4.052222
              125163869     2.038611
              233096537     0.398056
              441867932     3.739167
                              ...   
WeldingCorp2  9038020707    5.480556
              9072517040    1.129444
              9126734721    3.627778
              9389266951    8.659444
              9901183446    1.598056
Name: worked_hours, Length: 282, dtype: float64

In [6]:
workarea_tooltime = (cec_2018[pd.notnull(cec_2018['code_phase'])].
                          pivot_table(values='duration',index=['csas_id'],columns='code_phase',
                                      aggfunc='sum'))
workarea_tooltime['tool_time_pct'] = (workarea_tooltime['tool_time']/
                                  workarea_tooltime.iloc[:,0:5].sum(axis=1))
workarea_tooltime.head()

code_phase,down_time,idle_time,nonwork_time,tool_time,wait_time,tool_time_pct
csas_id,,,,,,
16064865,0.816667,1.283333,NaN,2.300000,0.066667,0.514925
47431840,0.250000,0.116667,4.750000,4.283333,0.100000,0.450877
125163869,1.116667,0.233333,NaN,7.633333,0.150000,0.835766
233096537,4.166667,0.066667,NaN,3.850000,0.100000,0.470468
389701276,1.000000,0.016667,0.416667,8.700000,0.233333,0.839228


In [7]:
workarea_tooltime.columns

Index(['down_time', 'idle_time', 'nonwork_time', 'tool_time', 'wait_time',
       'tool_time_pct'],
      dtype='object', name='code_phase')

In [8]:
company_score = company_score.merge(workarea_tooltime['tool_time_pct'],left_on='csas_id',right_on='csas_id',how='left')

In [9]:
company_score[['index_id','Company','worked_hours','tool_time_pct']].to_csv(path+"\\company_score.csv")

In [10]:
billable = company_score.groupby(['Company','clean_date','work_order'])['worked_hours'].sum().reset_index()

In [11]:
def get_billable(row):
    if (row['Company'] == 'LaborerCorp' or row['Company'] == 'MillwrightCorp1'):
        billable = row['worked_hours']*(1.1+.3*random.random())
    elif row['Company'] == 'WeldingCorp2':
        billable = row['worked_hours']*(1.34+.3*random.random())
    else:
        billable = row['worked_hours']*(1.2+.5*random.random())
    return billable

In [12]:
billable['billed_hours'] = billable.apply(get_billable,axis=1)

In [13]:
billable['Company'].unique()

array(['LaborerCorp', 'MillwrightCorp1', 'MillwrightCorp2',
       'WeldingCorp1', 'WeldingCorp2'], dtype=object)

In [14]:
billable.head()

,Company,clean_date,work_order,worked_hours,billed_hours
0,LaborerCorp,2018-05-26,21129601,1.000000,1.191840
1,LaborerCorp,2018-05-27,21129601,1.279444,1.661423
2,LaborerCorp,2018-05-28,21122364,1.202222,1.481307
3,LaborerCorp,2018-05-29,21098749,1.616667,1.829609
4,LaborerCorp,2018-05-29,21122364,1.373889,1.729566


In [15]:
billable.to_csv(path+"\\billable.csv")